# Extracting and processing STM Image Data from Omicron's propritary .Z_MTRX format 
In order to extract the images, first import the necessary packages. We will be using NanoscoPy to read the STM data and matplotlib to show the data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Custom modules
import sys
sys.path.append('../../NanoscoPy/')
from nanoscopy import spm
from nanoscopy.utilities import dialog_askdirectory

### Get Root Folder Path
Prompt the user for the directory which contains the .Z_mtrx files and the result file chain.

In [ ]:
folder_path = dialog_askdirectory()

### Read all STM files in the folder

In [ ]:
scans = spm.read(folder_path, metadata_source='filepath')

### Check metadata summary

In [ ]:
scans[0].summary()

### Collect forward trace images

In [ ]:
fwd_scans = [spm_data.fwd() for spm_data in scans]

### Correct the forward sweep data
Each image is globally flattened via plane correction, followed by 2nd order polynomial background subtraction, line-by-line offset flattening, 3x3 gaussian smoothing, and CLAHE.

In [ ]:
corrected_scans = spm.correct(fwd_scans, poly=True, equalize=True)

### Show a comparison plot between some of the raw and processed images

In [ ]:
n = len(corrected_scans[:5])
fig, axs = plt.subplots(n, 2)
fig.set_figheight(15)
fig.set_figwidth(7)

for i in range(n):
    raw = scans[i]
    processed = corrected_scans[i]
    sample_name = f'{raw.sample_id()}'
    # sample_name = 'Original'

    axs[i, 0].imshow(raw.fwd() , origin = 'lower')
    axs[i, 0].set_title(sample_name)
    axs[i, 1].imshow(processed , origin = 'lower')
    axs[i, 1].set_title(f'post-processed')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

### Output directories 

In [ ]:
dst_paths = [f'{Path(scan.path()).parent}\\{scan.summary()}.jpg' for scan in scans]
dst_paths

### Export the data as jpg

In [ ]:
spm.io.export_images(corrected_scans, dst_paths)

### Export metadata as csv 

In [ ]:
spm.io.export_metadata(scans, folder_path)